<a href="https://colab.research.google.com/github/gulmert89/dsBootcamp/blob/master/project3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 3: Classification Project
Data Set: [Twitter US Airline Sentiment](https://www.kaggle.com/crowdflower/twitter-airline-sentiment) <br>
*\"Analyze how travelers in February 2015 expressed their feelings on Twitter\"*

# 0 - Introduction

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [ ]:
# let's gather the gang for the party:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# import warnings

# warnings.filterwarnings("ignore")
%matplotlib inline
sns.set(style="whitegrid")

# specs of the party balloons
title_font = {"family":"serif", "weight":"bold", "color":"black", "size":20}
axis_font = {"family":"sans", "weight":"normal", "color":"black", "size":16}

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
# calling our honor guest:
tweets = pd.read_csv("/content/drive/My Drive/MertColab/proje3/tweets.csv")
tweets.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [ ]:
# first, she introduces herself
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14640 entries, 0 to 14639
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   tweet_id                      14640 non-null  int64  
 1   airline_sentiment             14640 non-null  object 
 2   airline_sentiment_confidence  14640 non-null  float64
 3   negativereason                9178 non-null   object 
 4   negativereason_confidence     10522 non-null  float64
 5   airline                       14640 non-null  object 
 6   airline_sentiment_gold        40 non-null     object 
 7   name                          14640 non-null  object 
 8   negativereason_gold           32 non-null     object 
 9   retweet_count                 14640 non-null  int64  
 10  text                          14640 non-null  object 
 11  tweet_coord                   1019 non-null   object 
 12  tweet_created                 14640 non-null  object 
 13  t

In [ ]:
# then we gaze upon her variables
nuniques = pd.DataFrame(columns=["Column Name", "Uniques #"])
for i in tweets.columns:
    nuniques = nuniques.append({"Column Name":i, "Uniques #":tweets[i].nunique()}, 
                                           ignore_index=True)
display(nuniques)
del(nuniques)

,Column Name,Uniques #
0,tweet_id,14485
1,airline_sentiment,3
2,airline_sentiment_confidence,1023
3,negativereason,10
4,negativereason_confidence,1410
5,airline,6
6,airline_sentiment_gold,3
7,name,7701
8,negativereason_gold,13
9,retweet_count,18


In [ ]:
# ...and she takes off her fancy fur coat and some other unnecessary ornaments 
tweets.drop(columns=["tweet_id", "airline_sentiment_gold", "name", 
                     "negativereason_gold", "retweet_count", "tweet_created", 
                     "tweet_coord", "tweet_created", "tweet_location", 
                     "user_timezone"], 
            inplace=True
            )
tweets.head()

,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,text
0,neutral,1.0000,NaN,NaN,Virgin America,@VirginAmerica What @dhepburn said.
1,positive,0.3486,NaN,0.0000,Virgin America,@VirginAmerica plus you've added commercials t...
2,neutral,0.6837,NaN,NaN,Virgin America,@VirginAmerica I didn't today... Must mean I n...
3,negative,1.0000,Bad Flight,0.7033,Virgin America,@VirginAmerica it's really aggressive to blast...
4,negative,1.0000,Can't Tell,1.0000,Virgin America,@VirginAmerica and it's a really big bad thing...


# 1 - Cleaning the dance area!

We are going to clean the tweets to only English words by removing:

* Symbols
* Numbers
* The airline brands (since we already have the corresponding column)

Now we need to find the Twitter account names of the airlines.

In [ ]:
import re, string, nltk
nltk.download('words')
words = set(nltk.corpus.words.words())

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


In [ ]:
# Airline companies:
list(tweets.airline.unique())

['Virgin America', 'United', 'Southwest', 'Delta', 'US Airways', 'American']

In [ ]:
# The twitter accounts mentioned:
set_them = set()
for i in tweets.text:
    search_them = re.search(r"(^|[^@\w])@(\w{1,15})\b", i)
    give_them = search_them.group().lower()
    set_them.add(give_them)

print(set_them)
del set_them

{'\n@jetblue', ' @southwestair', '@usairways', ',@usairways', '@vanessaannz', '“@virginamerica', '“@united', '"@jetblue', '@shannonwoodward', ' @virginamerica', '@nrhodes85', '.@americanair', '“@americanair', '@deltaassist', '@americanair', '@timbennettg3', '“@jetblue', '"@united', ' @chasefoster', '@hillaconlin', ' @nytimes', '@djevolutionhd', ' @united', '@malhoit', '@ninadavuluri', '@jetblue', ' @dadboner', '@united', ' @usairways', ' @melissaafrancis', '@sarahpompei', '@sb5551', ' @mandarinjourney', '“@usairways', '.@jetblue', '@eatgregeat', ' @kciairport', '.@virginamerica', ';@southwestair', '@virginamerica', '.@usairways', '@lindaswc', ' @americanair', '@catfoodbeerglue', '.@southwestair', '@southwestair', '@albertbreer', '.@united', ' @jetblue', ' @internjohnradio', '@andrewfallis', '@ods1819', ' @imaginedragons', '“@southwestair', '@scm1133'}


These are the Twitter accounts we will remove:
* @virginamerica
* @united
* @southwestair
* @deltaassist
* @usairways
* @americanair

In [ ]:
def tweet_cleaner(tweet):
    "Symbols, numbers & airline brand cleaner function!"
    # First step: Symbols and numbers are gone:
    cleaned_string = re.sub("[^a-zA-Z]", " ", tweet)

    # Party people here speak only English. No other languages are allowed:
    cleaned_string = " ".join(w for w in nltk.wordpunct_tokenize(cleaned_string) \
         if w.lower() in words)

    # Second step: Putting them to a list:
    cleaned_set = set(cleaned_string.lower().split())
    # Let's get rid of our meaningless one-letter friends: 
    what_to_clean = list(string.ascii_lowercase)
    # ...and some of their drunk buddies:
    what_to_clean.extend(["it", "of", "co", "to", "http"])
    # We don't need any officials in our party as well:
    what_to_clean.extend(["virginamerica", "united", 
                       "southwestair", "deltaassist",
                       "usairways", "americanair"])
    for delete in what_to_clean:
        try:
            cleaned_set.remove(delete)
        except:
            continue    
    return list(cleaned_set)

In [ ]:
# let's test the function we prepared for the party drinkers:
# example 1
print("Original tweet:\n", tweets.loc[100, "text"], "\n")
print("Cleaned and listed tweet:\n", tweet_cleaner(tweets.loc[100, "text"]))

Original tweet:
 @VirginAmerica trying to add my boy Prince to my ressie. SF this Thursday @VirginAmerica from LAX http://t.co/GsB2J3c4gM 

Cleaned and listed tweet:
 ['add', 'from', 'this', 'my', 'trying', 'lax', 'prince', 'boy']


In [ ]:
# example 2
print("Original tweet:\n", tweets.loc[2000, "text"], "\n")
print("Cleaned and listed tweet:\n", tweet_cleaner(tweets.loc[2000, "text"]))

Original tweet:
 @united wifi wasn't working onboard.Alerted attendant re socket.You sent me to a hotel for 24 hours with 7$ Vouchers??no wifi at hotel 

Cleaned and listed tweet:
 ['no', 'you', 'working', 'me', 'hotel', 'for', 'socket', 'sent', 'attendant', 'at', 'with', 're']


Good enough!<br>And one last thing:
We need to convert the "airline_sentiment" values to numbers to work on them easier. Here is what it's gonna be:

* "Neutral" sentiment becomes: 0
* "Positive" sentiment becomes: 1
* "Negative" sentiment becomes: 2

In [ ]:
print("Before:\n'Airline Sentiment' unique values:", tweets.airline_sentiment.unique())
for i,j in enumerate(tweets.airline_sentiment.unique()):
    tweets.airline_sentiment = tweets.airline_sentiment.replace(j, i)
tweets.airline_sentiment.astype("int64")
print("After:\n'Airline Sentiment' unique values:", tweets.airline_sentiment.unique())
# homework: please see the module ---> "sklearn.preprocessing.LabelEncoder().fit_transform(?)""

Before:
'Airline Sentiment' unique values: ['neutral' 'positive' 'negative']
After:
'Airline Sentiment' unique values: [0 1 2]


# 2 - Let the party begin!

We are going to collect all the words and give them binary inputs (1 or 0) according to their entities in the tweets.

In [ ]:
all_words = set()
rows=[] ### Tweetleri daha sonra aşağıda liste olarak kullanabilmek için
for i in tweets.index:
    listed_tweet = tweet_cleaner(tweets.text[i]) ### Aynı şeyi iki kez hesaplamamak için
    rows.append(listed_tweet) ### Tweetleri daha sonra aşağıda liste olarak kullanabilmek için
    all_words.update(listed_tweet) ### .add() yerine listeler için .update() kullanılabilir
print("Total number of distinct words:", len(all_words))

Total number of distinct words: 5361


In [ ]:
%%time
### Öncelikle, içi sıfırlarla dolu, sütun adları kelimeler olan ve
### indeksi tweet DF'nin indeksine eşit olan bir DF oluşturduk.
### Daha sonra satırları dolaşarak, satırlardaki kelimeleri içeren
### sütunların değerini değiştiriyoruz.

df_words = pd.DataFrame(0, columns=all_words, index=tweets.index)

for i in tweets.index:
    df_words.loc[i, rows[i]]=1

"""
df_words = pd.DataFrame()
for i in tweets.index:
    for j in all_words:
        condition = int(j in tweet_cleaner(tweets.loc[i, "text"]))
        df_words = df_words.append({j:condition}, ignore_index=True)
"""
### İç içe döngüler, kodların zaman karmaşıklığını inanılmaz derecede artırır.
### Şu anda olduğu gibi çok fazla satır ve sütun olduğunda gördüğümüz gibi işimiz oldukça uzun sürer.
### Benim yazdığım kod (satır bazında dolaşırken), benim bilgisayarımda 40,7sn sürdü.
### Bu durumda siz sütünları da dolaştığınıza göre, sizin kodunuzun tamamlanması
###      yine benim bilgisayarımda yaklaşık olarak (40,7 x 13624 = 554.497 sn) 6,4 gün sürer.
### Aradaki fark ilginç değil mi? :)
### Mümkün olduğunca iç içe döngülerden kaçınalım :)
print("The commented out code took tremendous amount of time. So it's reduced.")

The commented out code took tremendous amount of time. So it's reduced.
CPU times: user 8.89 s, sys: 306 ms, total: 9.2 s
Wall time: 9.21 s


In [ ]:
# it's time to concatenate the data frames
tweets = pd.concat([tweets, df_words], ignore_index=False, axis="columns")

## 2.1 - Logistic Regression Models

In [ ]:
from sklearn.model_selection import train_test_split, KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate, cross_val_score
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

X = tweets.iloc[:, 6:]
Y = tweets.airline_sentiment
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
%%time
lr_model1 = LogisticRegression(C=0.1, solver="lbfgs", max_iter=300, n_jobs=-1)
lr_model1.fit(x_train, y_train)
train_score_model1 = lr_model1.score(x_train, y_train)
test_score_model1 = lr_model1.score(x_test, y_test)

CPU times: user 1.37 s, sys: 538 ms, total: 1.91 s
Wall time: 38.6 s


In [ ]:
print("Model-1\nTrain score with 'lbfgs' solver: {:.3f}".format(train_score_model1))
print("Test score with 'lbfgs' solver: {:.3f}".format(test_score_model1))

Model-1
Train score with 'lbfgs' solver: 0.824
Test score with 'lbfgs' solver: 0.789


In [ ]:
%%time
lr_model2 = LogisticRegression(C=0.1, solver="liblinear", max_iter=300)
lr_model2.fit(x_train, y_train)
train_score_model2 = lr_model2.score(x_train, y_train)
test_score_model2 = lr_model2.score(x_test, y_test)

CPU times: user 1.66 s, sys: 127 ms, total: 1.79 s
Wall time: 1.49 s


In [ ]:
print("Model-2\nTrain score with 'liblinear' solver: {:.3f}".format(train_score_model2))
print("Test score with 'liblinear' solver: {:.3f}".format(test_score_model2))

Model-2
Train score with 'liblinear' solver: 0.807
Test score with 'liblinear' solver: 0.781


In [ ]:
# model 1 predictions:
train_predict_model1 = lr_model1.predict(x_train)
test_predict_model1 = lr_model1.predict(x_test)
predict_prob_model1 = lr_model1.predict_proba(x_test)[:, 1] # predict probability 
# model 2 predictions:
train_predict_model2 = lr_model2.predict(x_train)
test_predict_model2 = lr_model2.predict(x_test)
predict_prob_model2 = lr_model2.predict_proba(x_test)[:, 1]

In [ ]:
# You know we need to check how we did. Confusion matrix will help.
from sklearn.metrics import confusion_matrix, f1_score, classification_report
from sklearn.metrics import log_loss, accuracy_score
#from sklearn.metrics import precision_recall_fscore_support

# Model-1:
cm_train_model1 = confusion_matrix(y_train, train_predict_model1, 
                                   labels=[0, 1, 2])
cm_test_model1 = confusion_matrix(y_test, test_predict_model1, 
                                  labels=[0, 1, 2])
f1_score_model1 = f1_score(y_true=y_test, y_pred=test_predict_model1, 
                           average="weighted")    ## average= kısmını dokümandan okudum fakat tam anlamadım.
accuracy_score_model1 = accuracy_score(y_test, test_predict_model1)
#log_loss_model1 = log_loss(y_test, predict_prob_model1, labels=[0, 1, 2])
## HATA: ValueError: The number of classes in labels is different from that in y_pred. Classes found in labels: [0 1 2]

# Model-2:
cm_train_model2 = confusion_matrix(y_train, train_predict_model2, 
                                   labels=[0, 1, 2])
cm_test_model2 = confusion_matrix(y_test, test_predict_model2, 
                                  labels=[0, 1, 2])
f1_score_model2 = f1_score(y_true=y_test, y_pred=test_predict_model2, 
                           average="weighted")
accuracy_score_model2 = accuracy_score(y_test, test_predict_model2)
##log_loss_model2 = log_loss(y_test, predict_prob_model2, labels=[0, 1, 2])

In [ ]:
# Model-1:
print("MODEL-1:\nConfusion Matrix (Train Set)", "-"*30, cm_train_model1, sep="\n")
print("\nConfusion Matrix (Test Set)", "-"*30, cm_test_model1, sep="\n")
print(f"\nwith f1-score: {f1_score_model1:.3f}")
print(f"Accuracy score: {accuracy_score_model1:.3f}")
print(f"Error score: {(1-accuracy_score_model1):.3f}")
#print(f"Logarithmic Loss: {log_loss_model1:.3f}")
print("\n","*"*30, sep="")
# Model-2:
print("\nMODEL-2:\nConfusion Matrix (Train Set)", "-"*30, cm_train_model2, sep="\n")
print("Confusion Matrix (Test Set)", "-"*30, cm_test_model2, sep="\n")
print(f"with f1-score: {f1_score_model2:.3f}")
print(f"Accuracy score: {accuracy_score_model2:.3f}")
print(f"Error score: {(1-accuracy_score_model2):.3f}")
#print(f"Logarithmic Loss: {log_loss_model2:.3f}")

MODEL-1:
Confusion Matrix (Train Set)
------------------------------
[[1583  132  804]
 [ 226 1255  423]
 [ 360  121 6808]]

Confusion Matrix (Test Set)
------------------------------
[[ 315   45  220]
 [  60  285  114]
 [ 125   54 1710]]

with f1-score: 0.783
Accuracy score: 0.789
Error score: 0.211

******************************

MODEL-2:
Confusion Matrix (Train Set)
------------------------------
[[1415  140  964]
 [ 225 1189  490]
 [ 319  122 6848]]
Confusion Matrix (Test Set)
------------------------------
[[ 293   40  247]
 [  61  268  130]
 [ 114   49 1726]]
with f1-score: 0.772
Accuracy score: 0.781
Error score: 0.219


In [ ]:
print("Classification Report for Model-1:\n", classification_report(y_test, test_predict_model1))

Classification Report for Model-1:
               precision    recall  f1-score   support

           0       0.63      0.54      0.58       580
           1       0.74      0.62      0.68       459
           2       0.84      0.91      0.87      1889

    accuracy                           0.79      2928
   macro avg       0.74      0.69      0.71      2928
weighted avg       0.78      0.79      0.78      2928



In [ ]:
print("Classification Report for Model-2:\n", classification_report(y_test, test_predict_model2))

Classification Report for Model-2:
               precision    recall  f1-score   support

           0       0.63      0.51      0.56       580
           1       0.75      0.58      0.66       459
           2       0.82      0.91      0.86      1889

    accuracy                           0.78      2928
   macro avg       0.73      0.67      0.69      2928
weighted avg       0.77      0.78      0.77      2928



## 2.2 - Naive Bayes Models

In [ ]:
from sklearn.naive_bayes import MultinomialNB, GaussianNB

In [ ]:
%%time
naive = MultinomialNB()
parameters = {"alpha":np.logspace(-5,5,100), "fit_prior":[True, False]}
grid_cv = GridSearchCV(estimator=naive, param_grid=parameters, cv=10)
grid_cv.fit(x_train, y_train)
print("Best parameters : ", grid_cv.best_params_)
print("Best scores     : ", grid_cv.best_score_, "\n")

Best parameters :  {'alpha': 1.7886495290574351, 'fit_prior': False}
Best scores     :  0.7581099553195396 

CPU times: user 16min 36s, sys: 3min 21s, total: 19min 57s
Wall time: 15min 42s


In [ ]:
results = pd.DataFrame(grid_cv.cv_results_)
results = results[["param_alpha", "param_fit_prior", "mean_test_score"]]
results = results.sort_values(by="mean_test_score", ascending=False)
results.head(10)

,param_alpha,param_fit_prior,mean_test_score
105,1.78865,False,0.758110
103,1.41747,False,0.756830
101,1.12332,False,0.755294
107,2.25702,False,0.753585
94,0.559081,True,0.751878
96,0.70548,True,0.751451
92,0.443062,True,0.751451
98,0.890215,True,0.751195
90,0.351119,True,0.750853
100,1.12332,True,0.750427


In [ ]:
best_alpha = grid_cv.cv_results_["param_alpha"][105]
naive2 = MultinomialNB(alpha=best_alpha, fit_prior=False)
naive2.fit(x_train, y_train)

MultinomialNB(alpha=1.7886495290574351, class_prior=None, fit_prior=False)

In [ ]:
print("Test score:", naive2.score(x_test, y_test))

Test score: 0.7793715846994536


In [ ]:
naive3 = GaussianNB()
naive3.fit(x_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [ ]:
print("Test score:", naive3.score(x_test, y_test))

Test score: 0.3948087431693989


***
---
# Comments

**To do list:**

1. Tüm kelimeler kümelenerek DataFrame'e eklenecek.
2. Her bir index'e göre binary şekilde işaretlenecek.
3. Logistic regression modeli kurulacak.
    * Naive Bayes denenebilir.
4. Parametre ayarları yapılıp başarılı bir noktaya gelinecek.
5. Ardından şu iki seçenek denenerek model yeniden geliştirilecek.
    * "airline_sentiment_confidence" ve/veya "negativereason_confidence" değerleri belli bir değerden düşük olan (örn: 0.1 veya 0.2) datalar veri setinden çıkarılarak performansın değişimine bakılacak.
    * "tweet_cleaner" fonksiyonunda pasifleştirilen, İngilizce sözcükleri süzen satırlar aktifleştirilerek performansın değişimine bakılacak.
6. En başarılı model seçilerek partiye devam edilecek.
